## Prerequisite Knowledge
### 1. What is a neural network? High-level Concept vs Math
<img src="https://i.loli.net/2021/08/12/7yaGWTIKlHNz6mD.png" width='500'>

* High-level neural net (Left of the above image):  
    * **neural diagram** which is analogous to the biological network of neuron; multiple layers of **vertices/nodes/neurons** connected by **links**
    * Implementation Pro: helpful when you use `pytorch.nn` module or `keras` to build NN in high level (or layer level).
    * The activation of neurons in the previous layer determines the activation of ones in the current layer.


* Math  (Right of the above image): 
    * a series of transformation via **matrix-vector product** where transformation matrix is all the links between two layers in the neural diagram or 
    * **a function made up of a composition of multivirate, linear or non-linear activation functions** where weights are links and input/output values are neurons
    * Implementation Pro: helpful when you build neural network in a low level (or neuron level), e.g., `pytorch.functional` which contains the basic functions for high-level object-oriented modules with `torch.Tensor` (matrices) as parameters. it is also helpful when you 
        + want to trace the parameters and inputs/outputs of each layers
        + output flexible statistical estimates, 
        + design loss function without much numerical problem
        + understand and modify optimization process via calculus knowledge
 
    * From the high-school linear function to **multivirate functions**. Solving 
        $$2 w_{1}- w_{2}=0$$
        $$-w_{1} + 2w_{2}=3$$
        This could be concisely represented as matrix multipication:
        $$
        \left[\begin{array}{rr}
        2 & -1 \\
        -1 & 2
        \end{array}\right]\left[\begin{array}{l}
        x \\
        y
        \end{array}\right]=\left[\begin{array}{l}
        0 \\
        3
        \end{array}\right]
        $$
    Notice that: each equation represents a line in two dimensional space (or a plane) <-> For neural network, each unit represents the affine transformation (linear transformation with bias) of the whole plane since it contains a function rather than equation, e.g., $\text{hidden_neuron_1} = 2 w_{1}- w_{2}$

### 2. Common Transformations
* The common transformations could be divided into two categories: functions (no parameters during learning process) and architectures (containing learnable parameters). As a Pythonic framework for Deep Learning, PyTorch implements these two types explicitly as its standard libraries:
    * `torch.nn`: a standard library for PyTorch Out-of-the-box `Module`, e.g., `Linear`, `Conv1d`, `Dropout`
    * `torch.nn.functional`: a standard library for functions without associated parameters

* Linear Functioin 
    + Use case: basic transformation
    + Parameters: matrix $W$
    + Categories: Architecture
    + Notation: Let  $m$ = the number of examples, $n_i$ = the input dimension, $n_o$ = the output dimension
    + The linear function:  map input $X$ to the output $y$ by finding the parameters $W$ (determined by loss function): 
        +  $\hat{y} = W \cdot X$  where $\mathbf{X} \in \mathbb{R}^{n_i \times m}$ and $\mathbf{W} \in \mathbb{R}^{n_o \times n_i}$ and $\mathbf{\hat{Y}} \in \mathbb{R}^{n_o \times m}$ which is equivalent to 
        +  $\hat{y} = X \cdot W$  where $\mathbf{X} \in \mathbb{R}^{m \times n_i}$ and $\mathbf{W} \in \mathbb{R}^{n_i \times n_o}$ and $\mathbf{\hat{Y}} \in \mathbb{R}^{m \times n_o}$ (This form conforms to the implementation in Tensorflow).
        + If a bias term $b\in \mathbb{R}^{n_o \times 1}$ is added: $\hat{y} = W \cdot X + b$. But no need to do that since (1) $W$ do the job of transforming input or $b$ could be considered as  $w_{n+1} \dot x$ where $x=1$.
        + Although the first notation makes more sense for me to imagine it as the transformation in the high-dimension space, the second notation is commonly used for implementation. Here I change (use $\theta$ and transpose) the second notation into $\theta^{T} X$. Because $\theta$ which can avoid confusion for NLP notations and unify weights and bias.
        + Anyway, the linear node and bias nodes would not be shown in neural network diagrams since every neuron is assumed to have a linear node along with its corresponding bias. But we just use its output denoted as $z$ where $z = \theta^{T} X$
    
* Sigmoid, softmax, tanh
    $$p = h_\theta=\frac{1}{1+e^{-z}} $$
    + Use Case: output probability distribution
    + Parameters: outputs from the last layer
    + Categories: Functions
    
* Cross entropy for loss
    + Use case: Although this notebook is mostly relevant to forward pass of NN, another essence of neural network is learning parameters like $W$ vias optimization algorithm (normally gradient descend) which need well-defined [**objective/loss function**](https://www.kaggle.com/sergioli212/cross-entropy-loss-details).
    + Parameters: model outputs and ground truth
    + Categories: Functions

* More complication architectures： convolution (CNN), recurrent (RNN), Transformer
    + This is designed with utilizations of basic linear function and other functions.
    + World is built on space and time which introduce awesome properties into data(e.g. Pictures, Text, Audio,Vedio and sensor readings and so on). The most important property is that positions of one example matter to explain something. For example, the trend of temperature can only be reflected along sequential steps in order or images of human face could only be recognized when the nose is put between eyes and the mouth(otherwise, it would be recognized as the monster, actually nothing.) Even though these positional relations may be extracted by human, it just may not be well-thought-out and not intellgent when it comes to a bulk of data. Therefore, some NN architecture could be constructed to consider the nature of positional relations, e.g. Convolution NN and Recurrent NN. In essence, they are just another type of transformation or pieces of Lego for constructing more beautiful toys. Here I just take RNN for example. More details see: [RNN/LSTM](https://www.kaggle.com/sergioli212/rnn-lstm-from-scratch/), [CNN](https://www.kaggle.com/sergioli212/convolution/).


* (Optional) The way I think of why we need different transformations in neural net: Just like playing Lego we need more than regular pieces🔺 to build real-world things 🚉, 📱... , neural network consist of more than linear transformation to model the real-world problem. This depends on prior knowledge of what problem you want to solve. For example,  if you build ⏰, you need irregularly shaped pieces like wheel🏽⚙️. If you predict the increase of human population in China next year, you may use exponential function since you know  it will increase exponentially by time. If you predict image class, you will use convolution transformation since yoou know image has to be understood by its combination of pixels rather than each pixel separately.


### 3. Last words to make the story complete
* An Error Function is often used with a neural network for its appication, e.g.,  classification.
* With error function to quantify how wrong the current neural network, we can use gradient descent to improve the neural network by optimizing the weights/parameters/links. That's why we need a good error function.


In [ ]:
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable 
import tensorflow as tf
import matplotlib.pyplot as plt

## Basic data structure: `torch.Tensor`

In [ ]:
# matrix-vector product
W = torch.randn((4,3))
b = torch.randn(4)
x = torch.randn(3)
torch.matmul(W, x) + b

Use `Tensor` and primitive operations to build a linear layer from scratch

In [ ]:
# input data x with shape (10, 5)
X= torch.randn(10, 5)
# in numpy
# X = np.random.randn(10,5)

# Linear Function
W = torch.randn(2,10) # initialize parameters: W
b = torch.randn(2,1) # initialize parameters:  b
Z = torch.matmul(W, X) + b  
# equal to
# torch.mm(W, X) + b
# z_0 = torch.sum(W[0]* X[0].T) +b  # element-wise multiplication & sum for vector => matrix multiplication 
# in numpy
# W = np.random.randn(2,10) # initialize parameters: W
# b = np.random.randn(2,1) # initialize parameters:  b
# Z = np.dot(W, X)

Use `Tensor` and primitive operations to build a RNN layer from scratch

In [ ]:
# input data with middle dimention 10 represents sequence length
X = torch.Tensor(np.random.randn(10, 3, 4))
n_x, m, T_x = X.shape # 3, 10, 4
n_y, n_a = 2, 5 

# pytorch linear API + tanh activation function
linear_xa = torch.nn.Linear(n_x, n_a)
linear_aa = torch.nn.Linear(n_a, n_a)
a_prev = torch.randn(m, n_a)
for t in range(T_x):
    input1 = linear_xa(X[:,:,t])
    input2 = linear_aa(a_prev)
    a_prev = torch.tanh(input1+input2)
# print(a_prev.shape)

Let's see how neural network transforms images. I use
* a typical mnist dataset which aims to identify digits
*  two layers of Dense/fully connected layer.

In [ ]:
# Softmax Funtion
# def softmax(x, axis=0):
#     e_x = torch.exp(x - torch.max(x))
#     return e_x / e_x.sum(axis=axis)
def softmax(x, axis=-1):
    return torch.exp(x) / torch.sum(torch.exp(x), dim=axis).view(-1, 1)

# Sigmoid Function
# def sigmoid(x):
#     return 1/(1 + np.exp(-x)) 
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))


# To make generalized function for one layer with linear + activation function
def Layer(X, n_output, activation):
    n_input = X.shape[1]
    W = torch.randn(n_input, n_output)
    b = torch.randn((1, n_output))
    return activation(torch.mm(X, W) + b)
    

mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()
mnist_train = torch.utils.data.TensorDataset(torch.Tensor(mnist_train[0]),torch.Tensor(mnist_train[1]))
mnist_test = torch.utils.data.TensorDataset(torch.Tensor(mnist_test[0]),torch.Tensor(mnist_test[1]))
# shuffle the dataset every time we staet going through the data loader again, i.e. every iteration
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=64, shuffle=True) 
dataiter = iter(train_loader)
images, labels = dataiter.next()
# print(images.shape)
# print(labels.shape)
# plt.imshow(images[1].numpy(), cmap='Greys_r')
h = Layer(images.view(images.shape[0], -1), 256, sigmoid)
out = Layer(h, 10, softmax)

## High-level neural network using `nn.Module`

As said above, a neural network is just a stack of operations on data input tensors and model parameter tensors. `nn.Module` has the basic implementation to record the model parameters and operations in high level.

In a nutshell, 

**All the neural networks in Pytorch are built upon the parent class `nn.Module`**

The following code cell demonstrates how model parameters are used by Linear module `class Linear(Module)`.

In [ ]:
# Use Pytorch Linear Module
nn_module = nn.Linear(5, 2)
for p in nn_module.parameters():
    print('W or b: ', p.shape)

`Module` is used **in a nested way**.

In [ ]:
# build customized pytorch nn modules
class Network(nn.Module):
    def __init__(self):
        super().__init__() # pytorch will register layers and operations we put into the network
        
        self.hidden = nn.Linear(784, 256)
        self.output = nn.Linear(256, 10)
        
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.hidden(x)
        x = self.sigmoid(x)
        x = self.output(x)
        x = self.softmax(x)
        
        return x
        
model = Network()
model

## Closing Words 
* A neural network is an universal function approximator. For example, if you use softmax or sigmoid, a neural network represents conditional probability distribution. For understanding this, see [this notebook](https://www.kaggle.com/sergioli212/loss-function-and-optimization).
* the above models/functions are not trained so they are not a model fitting into any data. For understanding this, see [this notebook](https://www.kaggle.com/sergioli212/loss-function-and-optimization).
* Bullshit: The reason I would like to decompose NN architecture and modelling process is because, I think, I always see a lot of implementation packages and tutorials tend to mix up concepts or algorithms as a black box which hurts the flexibility to identify  and adjust finer pieces for solving my problem. For example, to generate adversarial text of NLP, I had a trouble to understand why we could update input instead of model parameters and what is problem of discrete text space and how to add constraints for ensuring fluency fo generating text via loss function (or adversarial objective functions here). All in all, each functions (linear, activation, stacking function, loss functions) during forward pass and their differentiation properties during backward pass should be well understood for deep learning practitioners.

## Practical Refereces
* [How to initialize weights?](https://stackoverflow.com/questions/49433936/how-to-initialize-weights-in-pytorch)